<a href="https://colab.research.google.com/github/forhigh/deeplearning_undergrad/blob/main/%5Bcomplete%5Dlab_07_6_FieldCompaction_ValidationDataset_EarlyStopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab_07_6 Deep Neural Nets
Field Compaction of Open-Graded Aggregates Validation Dataset
Early Stopping

환경 준비

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import pandas as pd
import numpy as np

데이터 준비, 정규화, Training/Testing 데이터셋 구분

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Data_set = np.loadtxt("/content/gdrive/My Drive/Colab Notebooks/FieldCompaction1.csv", delimiter=',', dtype=np.float32)
X = Data_set[:, 0:2]
Y = Data_set[:, 2:5]

# Normalize input dataset X
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

# Split dataset into training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


딥러닝 모델 구조 설정

In [3]:
model = Sequential()
model.add(Dense(10, input_dim=2, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 dense_2 (Dense)             (None, 3)                 18        
                                                                 
Total params: 103
Trainable params: 103
Non-trainable params: 0
_________________________________________________________________


모델 컴파일, 자동 중단 설정, 최적화 모델 저장 설정, 실행

In [4]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20) # 검증셋의 오차가 20회 이상 낮아지지 않을 경우 학습을 종료

# 최적화 모델이 저장될 폴더와 파일 이름 설정
modelpath = "/content/gdrive/My Drive/Colab Notebooks/Checkpoint/FieldCompaction-bestmodel.hdf5"

# 최적화 모델 저장 설정
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

# 모델 실행
history = model.fit(X_train, Y_train, epochs=5000, validation_split=0.25, verbose=0, callbacks=[early_stopping_callback,checkpointer])

테스트 결과 출력

In [5]:
score=model.evaluate(X_test, Y_test)
print('Test Loss:', score)

1/1 [==============================] - 0s 25ms/step - loss: 200.8824 - mean_squared_error: 200.8824
Test Loss: [200.88235473632812, 200.88235473632812]


history에 저장된 학습 결과 확인


In [6]:
hist_df=pd.DataFrame(history.history)
hist_df

,loss,mean_squared_error,val_loss,val_mean_squared_error
0,4991.188965,4991.188965,5253.104492,5253.104492
1,4990.056641,4990.056641,5251.912109,5251.912109
2,4988.905762,4988.905762,5250.713867,5250.713867
3,4987.705078,4987.705078,5249.507812,5249.507812
4,4986.528809,4986.528809,5248.282715,5248.282715
...,...,...,...,...
992,149.680725,149.680725,213.982864,213.982864
993,149.644196,149.644196,213.897003,213.897003
994,149.629196,149.629196,213.667114,213.667114
995,149.519516,149.519516,213.525543,213.525543
